In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

In [38]:
# Chargement des données
ins_prof_dut = pd.read_csv("../csv/fr-esr-insertion_professionnelle-dut_donnees_nationales.csv", sep=";")
ins_prof_lp = pd.read_csv("../csv//fr-esr-insertion_professionnelle-lp_donnees_nationales.csv", sep=";")
ins_prof_master = pd.read_csv("../csv/fr-esr-insertion_professionnelle-master_donnees_nationales.csv", sep=";")

In [39]:
# Agrégation de toutes les formations (DUT, LP, MASTER)
ins_prof = pd.concat([ins_prof_dut, ins_prof_lp, ins_prof_master], keys=["D", "L", "M"])

In [40]:
ins_prof.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5920 entries, ('D', 0) to ('M', 2924)
Data columns (total 86 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Année                                                                  5920 non-null   object 
 1   Diplôme                                                                5920 non-null   object 
 2   situation                                                              5920 non-null   object 
 3   Genre                                                                  5920 non-null   object 
 4   Disciplines                                                            5920 non-null   object 
 5   Code du domaine                                                        5920 non-null   object 
 6   Domaine                                                                592

In [41]:
ins_prof

Année     Diplôme                 situation             Genre  \
D 0     2013         DUT  30 mois après le diplôme            hommes   
  1     2017         DUT  18 mois après le diplôme  femmes et hommes   
  2     2017         DUT  18 mois après le diplôme            hommes   
  3     2017         DUT  18 mois après le diplôme  femmes et hommes   
  4     2017         DUT  18 mois après le diplôme            femmes   
...      ...         ...                       ...               ...   
M 2920  2013  MASTER ENS  18 mois après le diplôme            femmes   
  2921  2019  MASTER ENS  30 mois après le diplôme  femmes et hommes   
  2922  2015  MASTER ENS  30 mois après le diplôme            femmes   
  2923  2015  MASTER ENS  18 mois après le diplôme            hommes   
  2924  2017  MASTER ENS  30 mois après le diplôme            femmes   

                                              Disciplines Code du domaine  \
D 0     Droit, économie et gestion>Droit>Carrières jur...             DEG   
  1     Sciences, technologies et santé>Informatique>I...             STS   
  2     Sciences, technologies et santé>Informatique>I...             STS   
  3     Sciences humaines et sociales>Information comm...             SHS   
  4     Sciences humaines et sociales>Information comm...             SHS   
...                                                   ...             ...   
M 2920  Masters enseignement>Masters enseignement>Mast...            MEEF   
  2921  Masters enseignement>Masters enseignement>Mast...            MEEF   
  2922  Masters enseignement>Masters enseignement>Mast...            MEEF   
  2923  Masters enseignement>Masters enseignement>Mast...            MEEF   
  2924  Masters enseignement>Masters enseignement>Mast...            MEEF   

                                Domaine Code de la discipline  \
D 0          Droit, économie et gestion                disc02   
  1     Sciences, technologies et santé                disc16   
  2     Sciences, technologies et santé                disc16   
  3       Sciences humaines et sociales                disc10   
  4       Sciences humaines et sociales                disc10   
...                                 ...                   ...   
M 2920             Masters enseignement                disc18   
  2921             Masters enseignement                disc18   
  2922             Masters enseignement                disc18   
  2923             Masters enseignement                disc18   
  2924             Masters enseignement                disc18   

                       Discipline Sigle  ... Secteur disciplinaire  \
D 0                         Droit   NaN  ...                   NaN   
  1                  Informatique   NaN  ...                   NaN   
  2                  Informatique   NaN  ...                   NaN   
  3     Information communication   NaN  ...                   NaN   
  4     Information communication   NaN  ...                   NaN   
...                           ...   ...  ...                   ...   
M 2920       Masters enseignement   NaN  ...  Masters enseignement   
  2921       Masters enseignement   NaN  ...  Masters enseignement   
  2922       Masters enseignement   NaN  ...  Masters enseignement   
  2923       Masters enseignement   NaN  ...  Masters enseignement   
  2924       Masters enseignement   NaN  ...  Masters enseignement   

        Administration publique (hors ens.)  \
D 0                                     NaN   
  1                                     NaN   
  2                                     NaN   
  3                                     NaN   
  4                                     NaN   
...                                     ...   
M 2920                                  NaN   
  2921                                    2   
  2922                                  NaN   
  2923                                  NaN   
  2924                                  NaN   

       Prof. libérale, indépen

In [42]:
#ins_prof.set_index("Année", inplace=True)

In [43]:
# Liste des codes des domaines des diplômes
ins_prof["Code du domaine"].unique()

array(['DEG', 'STS', 'SHS', 'ALL', 'LLA', 'MEEF'], dtype=object)

In [44]:
# Délais d'insertion après diplôme
ins_prof["situation"].unique()

array(['30 mois après le diplôme', '18 mois après le diplôme'],
      dtype=object)

In [51]:
# Selection des colonnes des métiers + types de contrats
ins_prof.iloc[:, 30:85]

Prof. libérale, indépendant,' chef d’entreprise Fonctionnaire  CDI  \
D 0                                                NaN           NaN  NaN   
  1                                                  2             2   78   
  2                                                NaN           NaN  NaN   
  3                                                NaN           NaN  NaN   
  4                                                NaN           NaN  NaN   
...                                                ...           ...  ...   
M 2920                                             NaN           NaN  NaN   
  2921                                             NaN            76    8   
  2922                                             NaN           NaN  NaN   
  2923                                             NaN           NaN  NaN   
  2924                                             NaN           NaN  NaN   

       CDI de chantier ou CDI de mission Contrat spécifique au doctorat  CDD  \
D 0                                  NaN                            NaN  NaN   
  1                                  NaN                            NaN   14   
  2                                  NaN                            NaN  NaN   
  3                                  NaN                            NaN  NaN   
  4                                  NaN                            NaN  NaN   
...                                  ...                            ...  ...   
M 2920                               NaN                            NaN  NaN   
  2921                                 0                            NaN   14   
  2922                               NaN                            NaN  NaN   
  2923                               NaN                            NaN  NaN   
  2924                               NaN                            NaN  NaN   

       Vacataire Intérimaire Intermittent du spectacle  \
D 0          NaN         NaN                       NaN   
  1            0           2                         0   
  2          NaN         NaN                       NaN   
  3          NaN         NaN                       NaN   
  4          NaN         NaN                       NaN   
...          ...         ...                       ...   
M 2920       NaN         NaN                       NaN   
  2921         1           0                         0   
  2922       NaN         NaN                       NaN   
  2923       NaN         NaN                       NaN   
  2924       NaN         NaN                       NaN   

       Contrat de professionnalisa tion  ... Code du secteur disciplinaire  \
D 0                                 NaN  ...                           NaN   
  1                                   3  ...                           NaN   
  2                                 NaN  ...                           NaN   
  3                                 NaN  ...                           NaN   
  4                                 NaN  ...                           NaN   
...                                 ...  ...                           ...   
M 2920                              NaN  ...                        disc18   
  2921                              NaN  ...                        disc18   
  2922                              NaN  ...                        disc18   
  2923                              NaN  ...                        disc18   
  2924                              NaN  ...                        disc18   

       Secteur disciplinaire Administration publique (hors ens.)  \
D 0                      NaN                                 NaN   
  1                      NaN                                 NaN   
  2                      NaN                                 NaN   
  3                      NaN                                 NaN   
  4                      NaN                                 NaN   
...                      ...                                 ...   
M 2920  Mas